# Import packages

In [319]:
import pandas as pd

In [320]:
import plotly
import datapane as dp
plotly.offline.init_notebook_mode(connected=True)

In [321]:
import re
import pickle
import Levenshtein


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Upload json files

In [322]:
#files= ['0.json', '1.json', '2.json', '3.json', '4.json','42.json', '15693.json']
files = ['0.json']
# create an empty list to store the DataFrames
dfsList = [pd.read_json(file, orient='records') for file in files]

# combine all DataFrames into a single DataFrame
crossrefDF = pd.concat(dfsList, ignore_index=True)


In [323]:
crossrefDF

,items
0,{'URL': 'http://dx.doi.org/10.1053/eupc.2001.0...
1,"{'URL': 'http://dx.doi.org/10.1039/b110947b', ..."
2,{'URL': 'http://dx.doi.org/10.1093/rheumatolog...
3,{'URL': 'http://dx.doi.org/10.1080/01485010290...
4,{'abstract': '<jats:p>Explains the use of lear...
...,...
4995,"{'URL': 'http://dx.doi.org/10.1149/1.1391189',..."
4996,{'URL': 'http://dx.doi.org/10.1105/tpc.13.4.97...
4997,{'URL': 'http://dx.doi.org/10.1105/tpc.8.6.104...
4998,"{'URL': 'http://dx.doi.org/10.1149/1.1391821',..."


# Data preparation 

In [324]:
noAuthors = [i for i in range(len(crossrefDF)) if 'author' not in crossrefDF['items'][i]]

Authors = [i for i in range(len(crossrefDF)) if 'author'  in crossrefDF['items'][i]]

In [325]:
len(noAuthors) + len(Authors) == len(crossrefDF)

True

## Rows with authors

In [326]:
crossrefAuth = crossrefDF.iloc[Authors].copy()

crossrefAuth.reset_index(inplace= True)
crossrefAuth.drop(columns = ['index'], inplace = True)


## Extract 'DOI'

In [327]:
crossrefAuth.loc[:, 'DOI'] = crossrefAuth['items'].apply(lambda x: x['DOI'])

In [328]:
crossrefAuth.head()

,items,DOI
0,{'URL': 'http://dx.doi.org/10.1053/eupc.2001.0...,10.1053/eupc.2001.0161
1,"{'URL': 'http://dx.doi.org/10.1039/b110947b', ...",10.1039/b110947b
2,{'URL': 'http://dx.doi.org/10.1093/rheumatolog...,10.1093/rheumatology/41.7.824
3,{'URL': 'http://dx.doi.org/10.1080/01485010290...,10.1080/01485010290031565
4,{'abstract': '<jats:p>Explains the use of lear...,10.1108/02621719610120129


## Extract 'authors' --- number of authors

In [329]:
crossrefAuth.loc[:,'authors'] = crossrefAuth['items'].apply(lambda x: x['author'])


In [330]:
crossrefAuth.head()

,items,DOI,authors
0,{'URL': 'http://dx.doi.org/10.1053/eupc.2001.0...,10.1053/eupc.2001.0161,"[{'given': 'K', 'family': 'Kusano', 'sequence'..."
1,"{'URL': 'http://dx.doi.org/10.1039/b110947b', ...",10.1039/b110947b,"[{'given': 'Richard J.', 'family': 'Harrison',..."
2,{'URL': 'http://dx.doi.org/10.1093/rheumatolog...,10.1093/rheumatology/41.7.824,"[{'given': 'B. M.', 'family': 'Wroblewski', 's..."
3,{'URL': 'http://dx.doi.org/10.1080/01485010290...,10.1080/01485010290031565,"[{'given': 'C.-Y.', 'family': 'Lin', 'sequence..."
4,{'abstract': '<jats:p>Explains the use of lear...,10.1108/02621719610120129,"[{'given': 'Jean', 'family': 'Barclay', 'seque..."


In [331]:
numAuthors = [len(crossrefAuth.iloc[i]['authors']) for i in range(len(crossrefAuth))]

In [332]:
## yparxoun lathi  ---> kalytera number of affiliations
crossrefAuth.loc[:,'# authors'] = numAuthors

In [333]:
crossrefAuth.head()

,items,DOI,authors,# authors
0,{'URL': 'http://dx.doi.org/10.1053/eupc.2001.0...,10.1053/eupc.2001.0161,"[{'given': 'K', 'family': 'Kusano', 'sequence'...",1
1,"{'URL': 'http://dx.doi.org/10.1039/b110947b', ...",10.1039/b110947b,"[{'given': 'Richard J.', 'family': 'Harrison',...",3
2,{'URL': 'http://dx.doi.org/10.1093/rheumatolog...,10.1093/rheumatology/41.7.824,"[{'given': 'B. M.', 'family': 'Wroblewski', 's...",1
3,{'URL': 'http://dx.doi.org/10.1080/01485010290...,10.1080/01485010290031565,"[{'given': 'C.-Y.', 'family': 'Lin', 'sequence...",6
4,{'abstract': '<jats:p>Explains the use of lear...,10.1108/02621719610120129,"[{'given': 'Jean', 'family': 'Barclay', 'seque...",1


## Extract 'affiliations' --- number of affiliations

In [334]:
def getAff(k):
   return [crossrefAuth['authors'][k][j]['affiliation'] for j in range(len(crossrefAuth['authors'][k]))]
    

In [335]:
Affiliations = [getAff(k) for k in range(len(crossrefAuth))]

crossrefAuth.loc[:,'affiliations'] = Affiliations


In [336]:
numAffil = [len(Affiliations[i]) for i in range(len(crossrefAuth))]

In [337]:
crossrefAuth.loc[:,'# Affil'] = numAffil

In [338]:
crossrefAuth.head()

,items,DOI,authors,# authors,affiliations,# Affil
0,{'URL': 'http://dx.doi.org/10.1053/eupc.2001.0...,10.1053/eupc.2001.0161,"[{'given': 'K', 'family': 'Kusano', 'sequence'...",1,[[]],1
1,"{'URL': 'http://dx.doi.org/10.1039/b110947b', ...",10.1039/b110947b,"[{'given': 'Richard J.', 'family': 'Harrison',...",3,"[[], [], []]",3
2,{'URL': 'http://dx.doi.org/10.1093/rheumatolog...,10.1093/rheumatology/41.7.824,"[{'given': 'B. M.', 'family': 'Wroblewski', 's...",1,[[]],1
3,{'URL': 'http://dx.doi.org/10.1080/01485010290...,10.1080/01485010290031565,"[{'given': 'C.-Y.', 'family': 'Lin', 'sequence...",6,"[[], [], [], [], [], []]",6
4,{'abstract': '<jats:p>Explains the use of lear...,10.1108/02621719610120129,"[{'given': 'Jean', 'family': 'Barclay', 'seque...",1,[[]],1


## Clean 'empty' affiliations

In [339]:
possibleEmptyAff = []

for k in range(len(crossrefAuth)):
    if len(crossrefAuth['affiliations'][k][0]) == 0:
        possibleEmptyAff.append(k)

In [340]:
len(possibleEmptyAff)

3663

In [341]:
crossrefAuth['affiliations'].iloc[2611]

[[]]

In [342]:
nonEmptyAff = []

for k in possibleEmptyAff:
    for j in range(len(crossrefAuth['affiliations'].iloc[k])):
        if len(crossrefAuth['affiliations'].iloc[k][j]) != 0:
            nonEmptyAff.append(k)
    
    

In [343]:

FinalEmptyyAff=  [x for x in possibleEmptyAff if x not in nonEmptyAff] 

In [344]:
FinalNonEmptyAff = [x for x in range(len(crossrefAuth)) if x not in FinalEmptyyAff]

# affilDF: crossrefAuth subdataframe with nonpempty affiliation lists

In [345]:
affilDF = crossrefAuth.iloc[FinalNonEmptyAff].copy()
affilDF.reset_index(inplace = True)
affilDF.drop(columns = ['index'], inplace = True)

## (still some cleaning: cases with empty brackets [{}])

In [346]:
affilDF[affilDF['DOI'] == '10.48130/emst-2022-0020']

,items,DOI,authors,# authors,affiliations,# Affil


In [347]:
for k in range(len(affilDF)):
    if len(affilDF['affiliations'][k][0]) != 0 and affilDF['affiliations'][k][0][0] == {}:
        print(k)

In [348]:
emptyBrackets = [k for k in range(len(affilDF)) if len(affilDF['affiliations'][k][0]) != 0 and affilDF['affiliations'][k][0][0] == {}]

In [349]:
affilDF.iloc[emptyBrackets]

,items,DOI,authors,# authors,affiliations,# Affil


In [350]:
affilDF.copy()

,items,DOI,authors,# authors,affiliations,# Affil
0,{'URL': 'http://dx.doi.org/10.1105/tpc.9.9.167...,10.1105/tpc.9.9.1673,"[{'given': 'T', 'family': 'Nakano', 'sequence'...",6,[[{'name': 'Department of Agricultural Chemist...,6
1,{'URL': 'http://dx.doi.org/10.1164/rccm.210412...,10.1164/rccm.2104129,"[{'given': 'France', 'family': 'Gauvin', 'sequ...",8,[[{'name': 'Division of Pediatric Intensive Ca...,8
2,{'URL': 'http://dx.doi.org/10.1164/rccm.220402...,10.1164/rccm.2204029,"[{'given': 'Romain', 'family': 'Pauwels', 'seq...",1,[[{'name': 'Department of Respiratory Diseases...,1
3,{'URL': 'http://dx.doi.org/10.1164/rccm.220500...,10.1164/rccm.2205008,"[{'given': 'Laurent', 'family': 'Brochard', 's...",1,[[{'name': 'Henri Mondor Hospital University o...,1
4,{'abstract': '<jats:title>ABSTRACT</jats:title...,10.1128/iai.70.8.4494-4500.2002,"[{'given': 'Benfang', 'family': 'Lei', 'sequen...",8,[[{'name': 'Laboratory of Human Bacterial Path...,8
...,...,...,...,...,...,...
1131,{'URL': 'http://dx.doi.org/10.1105/tpc.8.11.19...,10.1105/tpc.8.11.1977,"[{'given': 'S E', 'family': 'Perry', 'sequence...",3,"[[{'name': 'Department of Botany, University o...",3
1132,{'abstract': '<jats:p> <jats:bold>...,10.1161/hs0402.105245,"[{'given': 'Luc', 'family': 'Djoussé', 'seque...",6,[[{'name': 'From the Section of Preventive Med...,6
1133,{'URL': 'http://dx.doi.org/10.1105/tpc.13.4.97...,10.1105/tpc.13.4.979,"[{'given': 'Mathias', 'family': 'Rossberg', 's...",8,[[{'name': 'Max-Delbrück-Laboratorium in der M...,8
1134,{'URL': 'http://dx.doi.org/10.1105/tpc.8.6.104...,10.1105/tpc.8.6.1041,"[{'given': 'M', 'family': 'Ni', 'sequence': 'f...",4,"[[{'name': 'Department of Plant Biology, Unive...",4


In [351]:
affilDF.drop(emptyBrackets, inplace = True)

In [352]:
affilDF.reset_index(inplace = True)

In [353]:
affilDF.copy()

,index,items,DOI,authors,# authors,affiliations,# Affil
0,0,{'URL': 'http://dx.doi.org/10.1105/tpc.9.9.167...,10.1105/tpc.9.9.1673,"[{'given': 'T', 'family': 'Nakano', 'sequence'...",6,[[{'name': 'Department of Agricultural Chemist...,6
1,1,{'URL': 'http://dx.doi.org/10.1164/rccm.210412...,10.1164/rccm.2104129,"[{'given': 'France', 'family': 'Gauvin', 'sequ...",8,[[{'name': 'Division of Pediatric Intensive Ca...,8
2,2,{'URL': 'http://dx.doi.org/10.1164/rccm.220402...,10.1164/rccm.2204029,"[{'given': 'Romain', 'family': 'Pauwels', 'seq...",1,[[{'name': 'Department of Respiratory Diseases...,1
3,3,{'URL': 'http://dx.doi.org/10.1164/rccm.220500...,10.1164/rccm.2205008,"[{'given': 'Laurent', 'family': 'Brochard', 's...",1,[[{'name': 'Henri Mondor Hospital University o...,1
4,4,{'abstract': '<jats:title>ABSTRACT</jats:title...,10.1128/iai.70.8.4494-4500.2002,"[{'given': 'Benfang', 'family': 'Lei', 'sequen...",8,[[{'name': 'Laboratory of Human Bacterial Path...,8
...,...,...,...,...,...,...,...
1131,1131,{'URL': 'http://dx.doi.org/10.1105/tpc.8.11.19...,10.1105/tpc.8.11.1977,"[{'given': 'S E', 'family': 'Perry', 'sequence...",3,"[[{'name': 'Department of Botany, University o...",3
1132,1132,{'abstract': '<jats:p> <jats:bold>...,10.1161/hs0402.105245,"[{'given': 'Luc', 'family': 'Djoussé', 'seque...",6,[[{'name': 'From the Section of Preventive Med...,6
1133,1133,{'URL': 'http://dx.doi.org/10.1105/tpc.13.4.97...,10.1105/tpc.13.4.979,"[{'given': 'Mathias', 'family': 'Rossberg', 's...",8,[[{'name': 'Max-Delbrück-Laboratorium in der M...,8
1134,1134,{'URL': 'http://dx.doi.org/10.1105/tpc.8.6.104...,10.1105/tpc.8.6.1041,"[{'given': 'M', 'family': 'Ni', 'sequence': 'f...",4,"[[{'name': 'Department of Plant Biology, Unive...",4


In [354]:
affilDF.drop(columns = ['index'], inplace = True)

# Clean affiliations 

## is_contained(a,b) map : returns true when a is a substring of b 

In [355]:
def is_contained(s, w):
    words = s.split()  # Split the string 's' into a list of words
    for word in words:
        if word not in w:  # If a word from 's' is not found in 'w'
            return False  # Return False immediately
    return True  # If all words from 's' are found in 'w', return True

## 1. "Unique" affiliations --- number of unique affiliations

In [356]:
uniqueAff = [] 

for i in range(len(affilDF)):
    uniqueAff.append(list(set([x[0] for x in [list(d.values()) for d in  [item for sublist in affilDF['affiliations'].iloc[i] for item in sublist]]])))


In [357]:
affilDF.loc[:,'uniqueAff'] = uniqueAff

In [358]:
numUniqueAff = [len(affilDF['uniqueAff'].iloc[i]) for i in range(len(affilDF))]

In [359]:
affilDF.loc[:,'# uniqueAff'] = numUniqueAff

In [360]:
affilDF.head()

,items,DOI,authors,# authors,affiliations,# Affil,uniqueAff,# uniqueAff
0,{'URL': 'http://dx.doi.org/10.1105/tpc.9.9.167...,10.1105/tpc.9.9.1673,"[{'given': 'T', 'family': 'Nakano', 'sequence'...",6,[[{'name': 'Department of Agricultural Chemist...,6,"[Department of Agricultural Chemistry, Faculty...",1
1,{'URL': 'http://dx.doi.org/10.1164/rccm.210412...,10.1164/rccm.2104129,"[{'given': 'France', 'family': 'Gauvin', 'sequ...",8,[[{'name': 'Division of Pediatric Intensive Ca...,8,[Division of Pediatric Intensive Care and Divi...,1
2,{'URL': 'http://dx.doi.org/10.1164/rccm.220402...,10.1164/rccm.2204029,"[{'given': 'Romain', 'family': 'Pauwels', 'seq...",1,[[{'name': 'Department of Respiratory Diseases...,1,[Department of Respiratory Diseases Ghent Univ...,1
3,{'URL': 'http://dx.doi.org/10.1164/rccm.220500...,10.1164/rccm.2205008,"[{'given': 'Laurent', 'family': 'Brochard', 's...",1,[[{'name': 'Henri Mondor Hospital University o...,1,[Henri Mondor Hospital University of Paris 12 ...,1
4,{'abstract': '<jats:title>ABSTRACT</jats:title...,10.1128/iai.70.8.4494-4500.2002,"[{'given': 'Benfang', 'family': 'Lei', 'sequen...",8,[[{'name': 'Laboratory of Human Bacterial Path...,8,"[Department of Pathology, Baylor College of Me...",2


## 2. Remove stop words ['from', 'the']

In [361]:
stopWords = ['from', 'the', 'From', 'The']

In [362]:
def remove_stop_words(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stopWords]
    return ' '.join(filtered_words)


# apply the function to the column  affilDF['uniqueAff'] to create column affilDF.loc[:,'uniqueAff1']

affilDF.loc[:,'uniqueAff1'] = affilDF['uniqueAff'].apply(lambda x: [remove_stop_words(s) for s in x])


## 3. Remove parenthesis 

In [363]:
def remove_parentheses(text):
   return re.sub(r'\([^()]*\)', '', text)

# apply the function to each list element of column affilDF['uniqueAff1'] to remove substrings inside parentheses

affilDF.loc[:,'uniqueAff1'] = affilDF['uniqueAff1'].apply(lambda x: [remove_parentheses(s) for s in x])


## 4. Remove @#$%characters 

In [364]:
affNoSymbols = []

for i in range(len(list(affilDF['uniqueAff1']))):
    L = list(affilDF['uniqueAff1'])[i]
    for i in range(len(L)):
        L[i] = re.sub(r'[^\w\s,Α-Ωα-ωぁ-んァ-ン一-龯，]', '', L[i])
    affNoSymbols.append(L)


In [365]:
affNoSymbols = [[item for item in inner_list if item != "inc"] for inner_list in affNoSymbols]


In [366]:
affilDF['uniqueAff1'] = affNoSymbols


## 5. Check 'sub'-affiliations (affiliations that are contained in other affiliations of the same DOI)

In [367]:
newAff0 = []

for k in range(len(affilDF)):
    
    L2 = []
    for s1 in affilDF['uniqueAff1'].iloc[k]:
        is_substring = False
        for s2 in affilDF['uniqueAff1'].iloc[k]:
            if s1 != s2 and s1 in s2:
                is_substring = True
                break
        if not is_substring:
            L2.append(s1)
    newAff0.append(L2)

In [368]:
newAffList = [list(set(newAff0[k])) for k in range(len(newAff0))]

In [369]:
affilDF['uniqueAff2'] = newAffList

In [370]:
affilDF.head(3)

,items,DOI,authors,# authors,affiliations,# Affil,uniqueAff,# uniqueAff,uniqueAff1,uniqueAff2
0,{'URL': 'http://dx.doi.org/10.1105/tpc.9.9.167...,10.1105/tpc.9.9.1673,"[{'given': 'T', 'family': 'Nakano', 'sequence'...",6,[[{'name': 'Department of Agricultural Chemist...,6,"[Department of Agricultural Chemistry, Faculty...",1,"[Department of Agricultural Chemistry, Faculty...","[Department of Agricultural Chemistry, Faculty..."
1,{'URL': 'http://dx.doi.org/10.1164/rccm.210412...,10.1164/rccm.2104129,"[{'given': 'France', 'family': 'Gauvin', 'sequ...",8,[[{'name': 'Division of Pediatric Intensive Ca...,8,[Division of Pediatric Intensive Care and Divi...,1,[Division of Pediatric Intensive Care and Divi...,[Division of Pediatric Intensive Care and Divi...
2,{'URL': 'http://dx.doi.org/10.1164/rccm.220402...,10.1164/rccm.2204029,"[{'given': 'Romain', 'family': 'Pauwels', 'seq...",1,[[{'name': 'Department of Respiratory Diseases...,1,[Department of Respiratory Diseases Ghent Univ...,1,[Department of Respiratory Diseases Ghent Univ...,[Department of Respiratory Diseases Ghent Univ...


## 6. Split strings where ',' or ';' appears    | Apply .lower()

In [371]:
import re

def substringsDict(string):
    split_strings = [re.sub(r'^[\s.]+|[\s.]+$', '', s.strip()) for s in re.split(r'[,;]', string)]
    dict_string = {}
    index = 0

    for value in split_strings:
        if value:
            dict_string[index] = value.lower()
            index += 1

    return dict_string


In [372]:
newAffkomma = []

for k in range(len(affilDF)):
    
    new_list = []
    for item in affilDF['uniqueAff2'].iloc[k]:
        new_list.append(substringsDict(item))


    newAffkomma.append(new_list)



In [373]:
removeList = ['university','research institute','laboratory' , 'universit','gmbh', 'inc', 'university of', 'research center', 
'university college','national institute of', 'school of medicine', "university school", 'graduate school of', 'graduate school of engineering', 
'institute of tropical medicine', 'institute of virology', 'faculty of medicine','laboratory', 'university park', 'institute of science']

city_names = ["Aberdeen", "Abilene", "Akron", "Albany", "Albuquerque", "Alexandria", "Allentown", "Amarillo", "Anaheim", "Anchorage", "Ann Arbor", "Antioch", "Apple Valley", "Appleton", "Arlington", "Arvada", "Asheville", "Athens", "Atlanta", "Atlantic City", "Augusta", "Aurora", "Austin", "Bakersfield", "Baltimore", "Barnstable", "Baton Rouge", "Beaumont", "Bel Air", "Bellevue", "Berkeley", "Bethlehem", "Billings", "Birmingham", "Bloomington", "Boise", "Boise City", "Bonita Springs", "Boston", "Boulder", "Bradenton", "Bremerton", "Bridgeport", "Brighton", "Brownsville", "Bryan", "Buffalo", "Burbank", "Burlington", "Cambridge", "Canton", "Cape Coral", "Carrollton", "Cary", "Cathedral City", "Cedar Rapids", "Champaign", "Chandler", "Charleston", "Charlotte", "Chattanooga", "Chesapeake", "Chicago", "Chula Vista", "Cincinnati", "Clarke County", "Clarksville", "Clearwater", "Cleveland", "College Station", "Colorado Springs", "Columbia", "Columbus", "Concord", "Coral Springs", "Corona", "Corpus Christi", "Costa Mesa", "Dallas", "Daly City", "Danbury", "Davenport", "Davidson County", "Dayton", "Daytona Beach", "Deltona", "Denton", "Denver", "Des Moines", "Detroit", "Downey", "Duluth", "Durham", "El Monte", "El Paso", "Elizabeth", "Elk Grove", "Elkhart", "Erie", "Escondido", "Eugene", "Evansville", "Fairfield", "Fargo", "Fayetteville", "Fitchburg", "Flint", "Fontana", "Fort Collins", "Fort Lauderdale", "Fort Smith", "Fort Walton Beach", "Fort Wayne", "Fort Worth", "Frederick", "Fremont", "Fresno", "Fullerton", "Gainesville", "Garden Grove", "Garland", "Gastonia", "Gilbert", "Glendale", "Grand Prairie", "Grand Rapids", "Grayslake", "Green Bay", "GreenBay", "Greensboro", "Greenville", "Gulfport-Biloxi", "Hagerstown", "Hampton", "Harlingen", "Harrisburg", "Hartford", "Havre de Grace", "Hayward", "Hemet", "Henderson", "Hesperia", "Hialeah", "Hickory", "High Point", "Hollywood", "Honolulu", "Houma", "Houston", "Howell", "Huntington", "Huntington Beach", "Huntsville", "Independence", "Indianapolis", "Inglewood", "Irvine", "Irving", "Jackson", "Jacksonville", "Jefferson", "Jersey City", "Johnson City", "Joliet", "Kailua", "Kalamazoo", "Kaneohe", "Kansas City", "Kennewick", "Kenosha", "Killeen", "Kissimmee", "Knoxville", "Lacey", "Lafayette", "Lake Charles", "Lakeland", "Lakewood", "Lancaster", "Lansing", "Laredo", "Las Cruces", "Las Vegas", "Layton", "Leominster", "Lewisville", "Lexington", "Lincoln", "Little Rock", "Long Beach", "Lorain", "Los Angeles", "Louisville", "Lowell", "Lubbock", "Macon", "Madison", "Manchester", "Marina", "Marysville", "McAllen", "McHenry", "Medford", "Melbourne", "Memphis", "Merced", "Mesa", "Mesquite", "Miami", "Milwaukee", "Minneapolis", "Miramar", "Mission Viejo", "Mobile", "Modesto", "Monroe", "Monterey", "Montgomery", "Moreno Valley", "Murfreesboro", "Murrieta", "Muskegon", "Myrtle Beach", "Naperville", "Naples", "Nashua", "Nashville", "New Bedford", "New Haven", "New London", "New Orleans", "New York", "New York City", "Newark", "Newburgh", "Newport News", "Norfolk", "Normal", "Norman", "North Charleston", "North Las Vegas", "North Port", "Norwalk", "Norwich", "Oakland", "Ocala", "Oceanside", "Odessa", "Ogden", "Oklahoma City", "Olathe", "Olympia", "Omaha", "Ontario", "Orange", "Orem", "Orlando", "Overland Park", "Oxnard", "Palm Bay", "Palm Springs", "Palmdale", "Panama City", "Pasadena", "Paterson", "Pembroke Pines", "Pensacola", "Peoria", "Philadelphia", "Phoenix", "Pittsburgh", "Plano", "Pomona", "Pompano Beach", "Port Arthur", "Port Orange", "Port Saint Lucie", "Port St. Lucie", "Portland", "Portsmouth", "Poughkeepsie", "Providence", "Provo", "Pueblo", "Punta Gorda", "Racine", "Raleigh", "Rancho Cucamonga", "Reading", "Redding", "Reno", "Richland", "Richmond", "Richmond County", "Riverside", "Roanoke", "Rochester", "Rockford", "Roseville", "Round Lake Beach", "Sacramento", "Saginaw", "Saint Louis", "Saint Paul", "Saint Petersburg", "Salem", "Salinas", "Salt Lake City", "San Antonio", "San Bernardino", "San Buenaventura", "San Diego", "San Francisco", "San Jose", "Santa Ana", "Santa Barbara", "Santa Clara", "Santa Clarita", "Santa Cruz", "Santa Maria", "Santa Rosa", "Sarasota", "Savannah", "Scottsdale", "Scranton", "Seaside", "Seattle", "Sebastian", "Shreveport", "Simi Valley", "Sioux City", "Sioux Falls", "South Bend", "South Lyon", "Spartanburg", "Spokane", "Springdale", "Springfield", "St. Louis", "St. Paul", "St. Petersburg", "Stamford", "Sterling Heights", "Stockton", "Sunnyvale", "Syracuse", "Tacoma", "Tallahassee", "Tampa", "Temecula", "Tempe", "Thornton", "Thousand Oaks", "Toledo", "Topeka", "Torrance", "Trenton", "Tucson", "Tulsa", "Tuscaloosa", "Tyler", "Utica", "Vallejo", "Vancouver", "Vero Beach", "Victorville", "Virginia Beach", "Visalia", "Waco", "Warren", "Washington", "Waterbury", "Waterloo", "West Covina", "West Valley City", "Westminster", "Wichita", "Wilmington", "Winston", "Winter Haven", "Worcester", "Yakima", "Yonkers", "York", "Youngstown"]

city_names = [x.lower() for x in city_names]

In [374]:
for j in range(len(newAffkomma)):
    for y in newAffkomma[j]:
        if len(y)>1:
            for i in range(len(y)-1):
                if (is_contained('lab', y[i]) or is_contained('inst', y[i]) or is_contained('hosp', y[i]) or is_contained('school', y[i])) and is_contained('univ', y[i+1]):
                    del y[i]
                elif is_contained('lab', y[i]) and (is_contained('college', y[i+1]) or is_contained('inst', y[i+1]) or is_contained('dep', y[i+1])):
                    del y[i]
                elif   is_contained('inst', y[i]) and (is_contained('dep', y[i+1]) or is_contained('acad', y[i+1]) or is_contained('hosp', y[i+1]) or is_contained('fac', y[i+1]) or is_contained('cent', y[i+1]) or is_contained('div', y[i+1])):
                    del y[i]
                elif y[i] in city_names+removeList:
                    del y[i]
          
            

In [375]:
affilDF['uniqueAff4'] =  [[list(d.values()) for d in sublist] for sublist in newAffkomma]


# Labels based on legalnames of openAIRE's organizations

In [376]:
uniList = ['institu', 'istituto','univ', 'college', 'center', 'centre' , 'cnrs', 'faculty','school' , 'academy' , 'école', 'hochschule' , 'ecole' ]

labList = ['lab']

hosplList = ['hospital' ,'clinic', 'hôpital']

gmbhList = ['gmbh', 'company' , 'industr', 'etaireia' , 'corporation', 'inc']

musList =  ['museum', 'library']

foundList =  ['foundation' , 'association','organization' ,'society', 'group' ]

deptList = ['district' , 'federation'  , 'government' , 'municipal' , 'county', 'ministry','council', 'agency']

unknownList = ['unknown']

#######   Dictionaries ##########

uniDict = {k: 'Univ/Inst' for k in uniList}   

labDict = {k: 'Laboratory' for k in labList} 

hosplDict = {k: 'Hospital' for k in hosplList}   

gmbhDict = {k: 'Company' for k in gmbhList}   

musDict = {k: 'Museum' for k in musList}   

#schoolDict = {k: 'School' for k in schoolList}   

foundDict = {k: 'Foundation' for k in foundList}   

deptDict = {k: 'Government' for k in deptList}   

unknownDict =  {k: 'Unknown' for k in unknownList}   

categDictsList = [uniDict, labDict, hosplDict, gmbhDict, musDict, #schoolDict, 
                  foundDict, deptDict, unknownDict]

################# Final Dictionary #####################

categDicts = {}
i = 0
while i in range(len(categDictsList)):
    categDicts.update(categDictsList[i])
    i = i+1
    
    


In [377]:
affilDF['uniqueAff4'].iloc[8]

[['department of electrical and computer engineering'],
 ['department of materials science and engineering',
  'north carolina state university',
  'north carolina 27695',
  'united states of america'],
 ['department of chemical engineering',
  'university of illinois at urbanachampaign',
  'urbana',
  'illinois 61801',
  'united states of america']]

## affiliationsDict

In [378]:
affiliationsDict = {}

for i in range(len(affilDF)):
    affiliationsDict[i] = affilDF['uniqueAff4'].iloc[i]
    


In [379]:
d_new = {}

# iterate over the keys of affiliationsDict
for k in range(len(affiliationsDict)):
    mappedk = []
    # get the list associated with the current key in affiliationsDict
    L = affiliationsDict.get(k, [])
    
    for x in L:
        mapped_listx = [[s, v] for s in x for k2, v in categDicts.items() if k2 in s]
        mappedk.append(mapped_listx)
    
    # add the mapped list to the new dictionary d_new
    d_new[k] = mappedk

In [380]:
affilDF['Dictionary'] = d_new

## What is not included 

In [381]:
affilDF['Dictionary'].iloc[5]

[[['colorado state university', 'Univ/Inst']]]

In [382]:
notInList = [i for i in range(len(affilDF)) if affilDF['Dictionary'].iloc[i] == [[]]]
    

In [383]:
affilDF.iloc[notInList]

,items,DOI,authors,# authors,affiliations,# Affil,uniqueAff,# uniqueAff,uniqueAff1,uniqueAff2,uniqueAff4,Dictionary
37,"{'abstract': '<jats:p> In this paper, we prese...",10.1142/s0218348x94000478,"[{'given': 'JACQUES LÉVY', 'family': 'VÉHEL', ...",3,"[[{'name': 'INRIA, Rocquencourt, B.P. 105, 781...",3,"[INRIA, Rocquencourt, B.P. 105, 78153 Le Chesn...",1,"[INRIA, Rocquencourt, BP 105, 78153 Le Chesnay...","[INRIA, Rocquencourt, BP 105, 78153 Le Chesnay...","[[inria, rocquencourt, bp 105, 78153 le chesna...",[[]]
48,{'abstract': '<jats:p>PAK1 is a protein kinase...,10.1161/01.res.0000014227.76102.5d,"[{'given': 'William B.', 'family': 'Kiosses', ...",7,[[{'name': 'From the Departments of Vascular B...,7,[From the Departments of Vascular Biology (W.B...,1,[Departments of Vascular Biology and Immunolo...,[Departments of Vascular Biology and Immunolo...,[[departments of vascular biology and immunol...,[[]]
137,{'abstract': '<jats:title>ABSTRACT</jats:title...,10.1128/aac.46.8.2640-2643.2002,"[{'given': 'Elizabeth', 'family': 'Pradel', 's...",2,"[[{'name': 'INSERM CJF9606-EA2197, IFR48, Facu...",2,"[INSERM CJF9606-EA2197, IFR48, Faculté de Mé...",1,"[INSERM CJF9606EA2197, IFR48, Faculte de Medec...","[INSERM CJF9606EA2197, IFR48, Faculte de Medec...","[[inserm cjf9606ea2197, ifr48, faculte de mede...",[[]]
158,{'abstract': '<jats:title>Abstract</jats:title...,10.1115/1.1456091,"[{'given': 'P. A.', 'family': 'Lakshminarayana...",4,"[[{'name': 'Kirloskar Oil Engines Ltd., Resear...",4,"[Kirloskar Oil Engines Ltd., Research and Engi...",1,"[Kirloskar Oil Engines Ltd, Research and Engin...","[Kirloskar Oil Engines Ltd, Research and Engin...","[[kirloskar oil engines ltd, research and engi...",[[]]
275,"{'URL': 'http://dx.doi.org/10.1021/cm0200792',...",10.1021/cm0200792,"[{'given': 'Chris', 'family': 'Blackman', 'seq...",7,"[[], [], [], [], [], [], [{'name': 'Epichem Li...",7,"[Epichem Limited, Power Road, Bromborough, Wir...",1,"[Epichem Limited, Power Road, Bromborough, Wir...","[Epichem Limited, Power Road, Bromborough, Wir...","[[epichem limited, power road, bromborough, wi...",[[]]
344,{'abstract': '<jats:title>Abstract</jats:title...,10.1104/pp.010916,"[{'given': 'Erika', 'family': 'Varkonyi-Gasic'...",2,"[[{'name': 'Plant Breeding and Genomics, AgRes...",2,"[Plant Breeding and Genomics, AgResearch, Priv...",1,"[Plant Breeding and Genomics, AgResearch, Priv...","[Plant Breeding and Genomics, AgResearch, Priv...","[[plant breeding and genomics, agresearch, pri...",[[]]
357,{'abstract': '<jats:p>It is suggested that ade...,10.1161/hy0901.095926,"[{'given': 'Undurti N.', 'family': 'Das', 'seq...",1,"[[{'name': 'EFA Sciences LLC, Norwood, Mass.'}]]",1,"[EFA Sciences LLC, Norwood, Mass.]",1,"[EFA Sciences LLC, Norwood, Mass]","[EFA Sciences LLC, Norwood, Mass]","[[efa sciences llc, norwood, mass]]",[[]]
403,{'URL': 'http://dx.doi.org/10.1080/09695940050...,10.1080/09695940050201398,"[{'given': 'Paul', 'family': 'Bennell', 'seque...",1,"[[{'name': 'a 60 Rugby Road, Brighton , B...",1,"[a 60 Rugby Road, Brighton , BN1 6ED , UK]",1,"[a 60 Rugby Road, Brighton , BN1 6ED , UK]","[a 60 Rugby Road, Brighton , BN1 6ED , UK]","[[a 60 rugby road, bn1 6ed, uk]]",[[]]
414,{'abstract': '<jats:p>`But my project is diffe...,10.1145/506218.506231,"[{'given': 'Robert L.', 'family': 'Glass', 'se...",1,[[{'name': 'publisher of the Software Practiot...,1,[publisher of the Software Practiotioner newsl...,1,[publisher of Software Practiotioner newslette...,[publisher of Software Practiotioner newslette...,[[publisher of software practiotioner newslett...,[[]]
426,{'URL': 'http://dx.doi.org/10.1105/tpc.8.11.19...,10.1105/tpc.8.11.1917,"[{'given': 'S', 'family': 'Somerville', 'seque...",2,[[{'name': 'Carnegie Institution of Washington...,2,"[Carnegie Institution of Washington, Departmen...",1,"[Carnegie Institution of Washington, Departmen...","[Carnegie Institution of Washington, Departmen...","[[department of plant biology, stanford, ca 94...",[[]]


In [384]:
len(affilDF)  - len(notInList)

1099

In [385]:
len(notInList)

37

# affilDF1 ['DOI', 'affiliations', 'Dictionary','uniqueAff4', 'uniqueAff2','# authors','# uniqueAff']

In [386]:
affilDF1 = affilDF[['DOI', 'affiliations','Dictionary','uniqueAff4', 'uniqueAff2','# authors','# uniqueAff']]

## New column: category based on the labels 

In [387]:
category = [', '.join(list(set([x[1] for y in affilDF1['Dictionary'].iloc[i] for x in y]))) for i in range(len(affilDF1))]
    


In [388]:
affilDF1 = affilDF1.copy()

In [389]:
affilDF1.loc[:, 'category'] = category


In [390]:
affilDF1.head(10)

,DOI,affiliations,Dictionary,uniqueAff4,uniqueAff2,# authors,# uniqueAff,category
0,10.1105/tpc.9.9.1673,[[{'name': 'Department of Agricultural Chemist...,"[[[faculty of agriculture, Univ/Inst], [kyoto ...","[[department of agricultural chemistry, facult...","[Department of Agricultural Chemistry, Faculty...",6,1,Univ/Inst
1,10.1164/rccm.2104129,[[{'name': 'Division of Pediatric Intensive Ca...,"[[[saintejustine hospital, Hospital], [univers...",[[division of pediatric intensive care and div...,[Division of Pediatric Intensive Care and Divi...,8,1,"Univ/Inst, Hospital"
2,10.1164/rccm.2204029,[[{'name': 'Department of Respiratory Diseases...,[[[department of respiratory diseases ghent un...,[[department of respiratory diseases ghent uni...,[Department of Respiratory Diseases Ghent Univ...,1,1,"Hospital, Univ/Inst"
3,10.1164/rccm.2205008,[[{'name': 'Henri Mondor Hospital University o...,[[[henri mondor hospital university of paris 1...,[[henri mondor hospital university of paris 12...,[Henri Mondor Hospital University of Paris 12 ...,1,1,"Hospital, Univ/Inst"
4,10.1128/iai.70.8.4494-4500.2002,[[{'name': 'Laboratory of Human Bacterial Path...,"[[[baylor college of medicine, Univ/Inst]], [[...","[[department of pathology, baylor college of m...","[Department of Pathology, Baylor College of Me...",8,2,"Laboratory, Univ/Inst"
5,10.1021/nl025567n,"[[{'name': 'Colorado State University, Departm...","[[[colorado state university, Univ/Inst]]]","[[colorado state university, department of che...","[Colorado State University, Department of Chem...",3,1,Univ/Inst
6,10.1161/hq0202.104001,[[{'name': 'From the Department of Medical Bio...,"[[[ehime university medical school, Univ/Inst]...","[[department of medical biochemistry, ehime un...","[Department of Medical Biochemistry, Ehime Uni...",7,1,Univ/Inst
7,10.1021/ma025508w,[[{'name': 'Departamento de Física de Material...,[[[and fundación donostia international physic...,[[departamento de física de materiales upvehu ...,[Departamento de Física de Materiales UPVEHU y...,3,1,Univ/Inst
8,10.1149/1.1392621,[[{'name': 'Department of Electrical and Compu...,"[[], [[north carolina state university, Univ/I...",[[department of electrical and computer engine...,[Department of Electrical and Computer Enginee...,4,3,Univ/Inst
9,10.1161/hc0202.102120,"[[{'name': 'From the Department of Medicine, C...",[[[university of washington school of medicine...,"[[department of medicine, cardiology division,...","[Department of Medicine, Cardiology Division, ...",2,1,Univ/Inst


### new label: rest

In [391]:
for i in range(len(affilDF1)):
    if affilDF1['category'].iloc[i] == [[]]:
        affilDF1.iloc[i, affilDF1.columns.get_loc('category')] = 'Rest'


In [392]:
affiliationsSimple = [
    list(set([inner_list[0] for outer_list in affilDF1['Dictionary'].iloc[i] for inner_list in outer_list]))
    for i in range(len(affilDF1))
]


In [393]:
affilDF1.loc[:,'affilSimple'] = affiliationsSimple

In [394]:
affilDF1.head()

,DOI,affiliations,Dictionary,uniqueAff4,uniqueAff2,# authors,# uniqueAff,category,affilSimple
0,10.1105/tpc.9.9.1673,[[{'name': 'Department of Agricultural Chemist...,"[[[faculty of agriculture, Univ/Inst], [kyoto ...","[[department of agricultural chemistry, facult...","[Department of Agricultural Chemistry, Faculty...",6,1,Univ/Inst,"[faculty of agriculture, kyoto university]"
1,10.1164/rccm.2104129,[[{'name': 'Division of Pediatric Intensive Ca...,"[[[saintejustine hospital, Hospital], [univers...",[[division of pediatric intensive care and div...,[Division of Pediatric Intensive Care and Divi...,8,1,"Univ/Inst, Hospital","[université de montréal, saintejustine hospital]"
2,10.1164/rccm.2204029,[[{'name': 'Department of Respiratory Diseases...,[[[department of respiratory diseases ghent un...,[[department of respiratory diseases ghent uni...,[Department of Respiratory Diseases Ghent Univ...,1,1,"Hospital, Univ/Inst",[department of respiratory diseases ghent univ...
3,10.1164/rccm.2205008,[[{'name': 'Henri Mondor Hospital University o...,[[[henri mondor hospital university of paris 1...,[[henri mondor hospital university of paris 12...,[Henri Mondor Hospital University of Paris 12 ...,1,1,"Hospital, Univ/Inst",[henri mondor hospital university of paris 12 ...
4,10.1128/iai.70.8.4494-4500.2002,[[{'name': 'Laboratory of Human Bacterial Path...,"[[[baylor college of medicine, Univ/Inst]], [[...","[[department of pathology, baylor college of m...","[Department of Pathology, Baylor College of Me...",8,2,"Laboratory, Univ/Inst","[national institutes of health, laboratory of ..."


In [395]:
len(affilDF1[affilDF1['category'] == 'Rest'])

0

# UNIVS & LABS

In [396]:
univLabs = [i for i in range(len(affilDF1)) if 'Laboratory' in affilDF1['category'].iloc[i] 
            or 'Univ/Inst' in  affilDF1['category'].iloc[i]]

In [397]:

univLabsDF = affilDF1.iloc[univLabs].copy()

In [398]:
univLabsDF.reset_index(inplace = True)

In [399]:
univLabsDF.drop(columns = ['index'], inplace = True)

# Load files from openAIRE

In [400]:
with open('dixOpenAIRE_Alletc.pkl', 'rb') as f:
    dixOpenAIRE_Alletc = pickle.load(f)


In [401]:
with open('dixOpenAIRE_id.pkl', 'rb') as f:
    dixOpenAIRE_id = pickle.load(f)


In [402]:
with open('dixOpenOrgId.pkl', 'rb') as f:
    dixOpenOrgId = pickle.load(f)


## Clean/modify the files

In [403]:
dixOpenAIRE_Alletc1 =  {k.replace(',', ''): v for k, v in dixOpenAIRE_Alletc.items()}

In [404]:
dixOpenAIRE_id1 = {k.replace(',', ''): v for k, v in dixOpenAIRE_id.items()}

In [405]:
dixOpenOrgId1 = {k.replace(',', ''): v for k, v in dixOpenOrgId.items()}

In [406]:
def filter_key(key):
    # Remove all non-alphanumeric characters except Greek letters and Chinese characters
    return re.sub(r'[^\w\s,Α-Ωα-ωぁ-んァ-ン一-龯，]', '', key)

def filter_dictionary_keys(dictionary):
    filtered_dict = {}
    for key, value in dictionary.items():
        filtered_key = filter_key(key)
        filtered_dict[filtered_key] = value
    return filtered_dict


dixOpenAIRE_Alletc1 = filter_dictionary_keys(dixOpenAIRE_Alletc1)
dixOpenAIRE_id1 = filter_dictionary_keys(dixOpenAIRE_id1)
dixOpenOrgId1 = filter_dictionary_keys(dixOpenOrgId1)



In [407]:
del dixOpenAIRE_Alletc1['laboratory']
del dixOpenAIRE_Alletc1['university hospital']

In [408]:
del dixOpenAIRE_id1['laboratory']
del dixOpenAIRE_id1['university hospital']

In [409]:
del dixOpenOrgId1['university hospital']

# MATCHINGS

## Helper functions

### Clean the matchings

In [412]:
def bestSimScore(l1, l2, l3):
    """
    Finds the best match between a 'key word' and several legal names from the OpenAIRE database.
    ---> corrects special cases in the main map that follows

    Args:
        l1: List of unique affiliations.
        l2: List of matched OpenAIRE names.
        l3: List of pairs.

    Returns:
        List: Resulting list containing OpenAIRE names and their similarity scores.
    """
    
    vectorizer = CountVectorizer()
    numUniv = sum([(l1[i].lower()).count('univ') for i in range(len(l1))])
    result = []
    for i in range(len(l1)):
        best = [] 
        s = l1[i]
       # s_contains_university = is_contained("university", s.lower())  
        
        if not is_contained("univ", s.lower()):
            continue  # Skip if s does not contain "university" or "univ"
        
    
        for j in range(len(l2)):
            x = l2[j]  
              
            if l3[j][2] >=0.99 and (is_contained("univ", x.lower()) or is_contained("college", x.lower()) or  is_contained("center", x.lower())): # If the similarity score of a pair (s,x) was 1, we store it to results list
                result.append([l2[j], 1])
                
            else:
        #        x_contains_university = is_contained("university", x.lower())
                if not is_contained("univ", x.lower()):
                    continue  # Skip if x does not contain "university" or "univ"
                
            
                s_vector = vectorizer.fit_transform([s]).toarray() #Else we compute the similarity of s with the original affiiation name
                x_vector = vectorizer.transform([x]).toarray()
                s_vector1 =  vectorizer.transform([s]).toarray()
                x_vector1 =  vectorizer.fit_transform([s]).toarray()
                

                # Compute similarity between the vectors
                similarity = cosine_similarity(x_vector, s_vector)[0][0]
                similarity1 = cosine_similarity(x_vector1, s_vector1)[0][0]
                similarity2 = Levenshtein.ratio(s,x)


                best.append([l2[j], (similarity+similarity1+similarity2)/3])
        
        if best:
            max_score = max(best, key=lambda x: x[1])[1]
            max_results = [(x[0], x[1]) for x in best if x[1] == max_score]
            if len(max_results) > 1:
                max_results.sort(key=lambda x: (l2.index(x[0]), -x[1]), reverse=False)
                result.append(max_results[-1])
            else:
                result.append(max_results[0])
                
    univ_list = []
    other_list = []
    
    for r in result:
        if is_contained('univ',r[0]):
            univ_list.append(r)
        else:
            other_list.append(r)
            
    if len(univ_list)> numUniv:
        result = univ_list[:numUniv] + other_list
                
    return result

### Find rows with multiple mathcings

In [413]:
def index_multipleMatchings(df):
    multipleMatchings = []

    for i in range(len(df)):
        result_dict = {}

        for t in [t[0] for t in df.Pairs.iloc[i]]:
            key = t
            if key in result_dict:
                result_dict[key] += 1
                multipleMatchings.append(i)
                
            else:
                result_dict[key] = 1
    return list(set(multipleMatchings))
                
        

## Main map

In [414]:
def Doi_Ids(m, DF, dixOpenAIRE, simU, simG):
    
    """
    Matches affiliations in DataFrame 'DF' with names from dictionary 'dixOpenAIRE' and their openAIRE ids based on similarity scores.

    Args:
        m (int): The number of DOIs to check.
        DF (DataFrame): The input DataFrame containing affiliation data.
        dixOpenAIRE (dict): A dictionary of names from OpenAIRE.
        simU (float): Similarity threshold for universities.
        simG (float): Similarity threshold for non-universities.

    Returns:
        DataFrame: The final DataFrame with matched affiliations and their corresponding similarity scores.
    """
    
    lnamelist = list(dixOpenAIRE.keys())
    dix = {}    # will store indeces and legalnames of organizations of the DOI { i : [legalname1, legalname2,...]}
    deiktes = []  # stores indeces where a match is found
    vectorizer = CountVectorizer()
    similarity_ab = [] # stores lists of similarity scores of the mathces 
    pairs = [] #  pairs[i] =  [ [s,x,t] ] where (s,x) is a match and t the corresponding similarity score
    
    for k in range(m):
        similar_k = []
        pairs_k = []
        numMathcings_k =[]


        for s in DF['affilSimple'].iloc[k]:
            count_s = 0

            if s in lnamelist:
                deiktes.append(k)
                similarity = 1
                similar_k.append(similarity)
                
                pairs_k.append((s,s,similarity))
                count_s += 1

                if k not in dix:
                    dix[k] = [s]
                else:
                    dix[k].append(s)
            else:

                for x in lnamelist:
                    
                    if  is_contained(s, x):
                        x_vector = vectorizer.fit_transform([x]).toarray()
                        s_vector = vectorizer.transform([s]).toarray()

                        # Compute similarity between the vectors
                        similarity = cosine_similarity(x_vector, s_vector)[0][0]
                        if similarity > min(simU, simG):
                            if (is_contained('univ', s) and is_contained('univ', x)) and similarity > simU:
                                similar_k.append(similarity)
                                deiktes.append(k)
                                pairs_k.append((s,x,similarity))
                                count_s = count_s+1


                                if k not in dix:
                                    dix[k] = [x]
                                else:
                                    dix[k].append(x)
                            elif (not is_contained('univ', s) and not is_contained('univ', x)) and similarity > simG:
                                similar_k.append(similarity)
                                deiktes.append(k)
                                pairs_k.append((s,x,similarity))
                                count_s += 1


                                if k not in dix:
                                    dix[k] = [x]
                                else:
                                    dix[k].append(x)
                    elif is_contained(x, s):
                        if (is_contained('univ', s) and is_contained('univ', x)):

                            if 'and' in s:
                                list_s = s.split(' and ')
                                for t in list_s:
                                    if is_contained(x, t) and is_contained('univ', t):
                                        t_vector = vectorizer.fit_transform([t]).toarray()
                                        x_vector = vectorizer.transform([x]).toarray()

                            # Compute similarity between the vectors
                                        similarity = cosine_similarity(t_vector, x_vector)[0][0]
                                        if similarity > simU:
                                            similar_k.append(similarity)
                                            deiktes.append(k)
                                            pairs_k.append((s,x,similarity))
                                            count_s += 1



                                            if k not in dix:
                                                dix[k] = [x]
                                            else:
                                                dix[k].append(x)
                            
                            else: 
                                s_vector = vectorizer.fit_transform([s]).toarray()
                                x_vector = vectorizer.transform([x]).toarray()

                                # Compute similarity between the vectors
                                similarity = cosine_similarity(s_vector, x_vector)[0][0]
                                if similarity > 0.81: #max(0.82,sim):
                                    similar_k.append(similarity)
                                    deiktes.append(k)
                                    pairs_k.append((s,x,similarity))
                                    count_s += 1 



                                    if k not in dix:
                                        dix[k] = [x]
                                    else:
                                        dix[k].append(x)
                        elif not is_contained('univ', s) and not is_contained('univ', x):
                            if 'and' in s:
                                list_s = s.split(' and ')
                                for t in list_s:
                                    if is_contained(x, t):
                                        t_vector = vectorizer.fit_transform([t]).toarray()
                                        x_vector = vectorizer.transform([x]).toarray()

                            # Compute similarity between the vectors
                                        similarity = cosine_similarity(t_vector, x_vector)[0][0]
                                        if similarity > simG:
                                            similar_k.append(similarity)
                                            deiktes.append(k)
                                            pairs_k.append((s,x,similarity))
                                            count_s +=1



                                            if k not in dix:
                                                dix[k] = [x]
                                            else:
                                                dix[k].append(x)
                            
                            else: 
                                s_vector = vectorizer.fit_transform([s]).toarray()
                                x_vector = vectorizer.transform([x]).toarray()

                                # Compute similarity between the vectors
                                similarity = cosine_similarity(s_vector, x_vector)[0][0]
                                if similarity > simG: #max(0.82,sim):
                                    similar_k.append(similarity)
                                    deiktes.append(k)
                                    pairs_k.append((s,x,similarity))
                                    count_s +=1

                                    if k not in dix:
                                        dix[k] = [x]
                                    else:
                                        dix[k].append(x)
                            
        similarity_ab.append(similar_k)   
        similarity_ab = [lst for lst in similarity_ab if lst != []]
        pairs.append(pairs_k)
        
    perc = 100*len(dix)/m
    
    dixDoiAff = {DF['DOI'].iloc[key]: value for key, value in dix.items()} # dictionary {DOI : legalnames} 
    
    #dixDoiPid1 = {key : [dixOpenAIRE[x] for x in value if x in  lnamelist] for key , value in dixDoiAff.items()}
    
   # dixDoiPid = {key : [dixOpenAIRE[x] for x in value] for key , value in dixDoiAff.items()} # dictionary {DOI : PIDs} 
    
    
    
## Define the new Dataframe
    
    doiIdDF = pd.DataFrame()
    doiIdDF['DOI'] = list(dixDoiAff.keys())
    doiIdDF['Affiliations'] = list(DF['affiliations'].iloc[list(set(deiktes))])

    doiIdDF['Unique affiliations'] = list(DF['uniqueAff2'].iloc[list(set(deiktes))])
    
    doiIdDF['# Authors'] = list(DF['# authors'].iloc[list(set(deiktes))])


    doiIdDF['# Unique affiliations'] = list(DF['# uniqueAff'].iloc[list(set(deiktes))])

    doiIdDF['Candidates for matching'] = list(DF['affilSimple'].iloc[list(set(deiktes))])

    doiIdDF['Matched openAIRE names'] = list(dix.values())
    doiIdDF['# Matched orgs'] = [len(list(dix.values())[i]) for i in range(len(list(dix.values())))]
    

    doiIdDF['Similarity score'] = similarity_ab
    perfectSim = [[1 if num >= 1 else 0 for num in inner_list] for inner_list in similarity_ab]

    doiIdDF['Perfect match'] = perfectSim
    perfectSimSum = [sum(x) for x in perfectSim]
    doiIdDF['Perfect sum'] = perfectSimSum
    Pairs = [lst for lst in pairs if lst]
    doiIdDF['Pairs'] = Pairs
    


## Correct the matchings
    
    needCheck = [i for i  in range(len(doiIdDF)) if doiIdDF['# Matched orgs'].iloc[i] - max(doiIdDF['# Authors'].iloc[i],doiIdDF['# Unique affiliations'].iloc[i]) >0 or    i in index_multipleMatchings(doiIdDF)]

    ready = [i for i in range(len(doiIdDF)) if i not in needCheck]
   
    best = [ bestSimScore(doiIdDF['Unique affiliations'].iloc[i], doiIdDF['Matched openAIRE names'].iloc[i], doiIdDF['Pairs'].iloc[i]) for i in needCheck]
    best_o = []
    best_s = []
    
    for x in best:
        best_o.append([x[i][0]  for i in range(len(x))])
        best_s.append([round(x[i][1],2)  for i in range(len(x))])
    numMathced = [len(best_s[i]) for i in range(len(needCheck))]
    

    
    dfFinal0 = (doiIdDF.iloc[ready]).copy()
    dfFinal0['index'] = ready
    
    dfFinal1 = (doiIdDF.iloc[needCheck]).copy()
    dfFinal1['index'] = needCheck
    dfFinal1['Matched openAIRE names'] = best_o
    dfFinal1['Similarity score'] = best_s
    dfFinal1['# Matched orgs'] = numMathced
    
    finalDF =  pd.concat([dfFinal0, dfFinal1])
    finalDF.set_index('index', inplace=True)
    finalDF.sort_values('index', ascending=True, inplace = True)
    
    ids = [[dixOpenAIRE[x] for x in v] for v in finalDF['Matched openAIRE names']]
    numIds = [len(x) for x in ids]

    finalDF['IDs'] = ids
    finalDF['# IDs'] = numIds
# Assuming you want to remove rows where the 'column_name' meets a specific condition
    finalDF = finalDF[~(finalDF['# Matched orgs'] == 0)]
    finalDF = finalDF.reset_index(drop=True)


    
    
    return [perc, finalDF, doiIdDF, needCheck]
    


In [415]:
result = Doi_Ids(len(univLabsDF), univLabsDF, dixOpenOrgId1, 0.7,0.82)

# HTML

In [ ]:
mydf = result[1]
myDF = mydf[["DOI", "Unique affiliations",'# Authors','# Unique affiliations', '# Matched orgs','Candidates for matching', 'Matched openAIRE names', 'Similarity score']]


In [ ]:
def update_Z(row):
    new_Z = []
    for i in range(len(row['IDs'])):
        entry = {'openaireId': row['IDs'][i], 'confidence': row['Similarity score'][i]}
        new_Z.append(entry)
    return new_Z

# Update the values in column 'Z' using 'apply'
mydf['affiliations'] = mydf.apply(update_Z, axis=1)


In [ ]:
myDF_output = mydf[['DOI','affiliations']]
myDF_output = myDF_output.rename(columns={'DOI': 'doi'}).copy()



In [ ]:
len(mydf)

675

In [ ]:
app = dp.App(dp.Page(title="Matchings [explained]", blocks= [dp.Text('Matched '+str(result[0])+'%'), dp.Text('DOIs with aff. in Univ/Inst, Laboratories'), dp.Table(myDF)]
             ), 
           #  dp.Page(title="Final matchings", blocks= [dp.Text('Matched '+str(result[0])+'%'), dp.Text('DOIs with aff. in Univ/Inst, Laboratories'), dp.Table(myDF_output)]
             #)
             )
    


   
app.save(path="test.html", open=True)

App saved to ./test.html

# JSON

In [ ]:
match0 = myDF_output.to_json(orient='records')

# Save the JSON to a file
with open('match0.json', 'w') as f:
    f.write(match0)